source API URL : "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

JSON Target File Path : "abfss://bronze@adlsalidatalakehousedev.dfs.core.windows.net/geo-location/"

In [0]:
geoLocationSourceAPIURL = "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

geoLocationSourceAPIBaseURL = "https://geocoding-api.open-meteo.com/v1/search?name="
geoLocationSourceAPIURLOptions = "&count=10&language=en&format=json"

geoLocationSinkLayerName = 'bronze'
geoLocationSinkStorageAccountName = 'adlsalidatalakehousedev'
geoLocationSinkFolderName = 'geo-location'

geoLocationSinkFolderPath = f"abfss://{geoLocationSinkLayerName}@{geoLocationSinkStorageAccountName}.dfs.core.windows.net/{geoLocationSinkFolderName}"

In [0]:
import requests
import json
import pandas as pds

In [0]:
geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()
geoLocationPandasDF = pds.DataFrame(geoLocationAPIResponse)
geoLocationSparkDF = spark.createDataFrame(geoLocationPandasDF)

In [0]:
dailyPricingMarketNamesDF = spark.sql("SELECT MARKET_NAME from pricing_analytics.gold.reporting_dim_market_gold")

In [0]:
marketNames = [dailyPricingMarketNames["MARKET_NAME"] for dailyPricingMarketNames in dailyPricingMarketNamesDF.collect()]
geoLocationAPIResponseList=[]

for marketName in marketNames:
    geoLocationSourceAPIURL = f"{geoLocationSourceAPIBaseURL}{marketName}{geoLocationSourceAPIURLOptions}"
    geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL,timeout=10).json()
    if isinstance(geoLocationAPIResponse, dict): #if null, not need to append
        geoLocationAPIResponseList.append(geoLocationAPIResponse)
    
geoLocationPandasDF = pds.DataFrame(geoLocationAPIResponseList)

geoLocationSparkDF = spark.createDataFrame(geoLocationPandasDF)
(geoLocationSparkDF
    .filter("results.admin1 is not null")
     .write
     .mode("overwrite")
     .json(geoLocationSinkFolderPath)
    )

In [0]:
geoLocationBronzeDF = spark.read.json(geoLocationSinkFolderPath)
display(geoLocationBronzeDF)